In [1]:
#Match_home_team_info table

from glob import glob
import pandas as pd

# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')
column_home= ['match_id','name']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [2]:
#match_away_team_info table
column_away= ['match_id','name']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_away]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [3]:
# Match_event_info table
files_event = []
column_event= ['match_id','winner_code']
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_event.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/event*.parquet')
    )
event_info = [pd.read_parquet(file, columns=column_event) for file in files_event]
event_info = pd.concat(event_info, ignore_index=True)

In [5]:
#Which player has the highest number of wins? 

df_home = event_info.merge(df_home_team_info, on='match_id',how='left')
df = event_info.merge(df_home, on='match_id',how='left')
df = df.drop(['winner_code_x'],axis=1)
df=df.rename(columns={'winner_code_y':'winner_code'})
#count home_team wins
home=df[df['winner_code']==1].groupby(['name','winner_code'])['winner_code'].count()
#count away_team wins
away=df[df['winner_code']==2].groupby(['name','winner_code'])['winner_code'].count()
#player who wons the most in all matches
total_wins = home.add(away, fill_value=0)
top_player = total_wins.idxmax()
max_wins = total_wins.max()

print(f"The player with the highest number of wins is {top_player[0]} with {int(max_wins)} wins.")

The player with the highest number of wins is Aguilar Cardozo J. with 220 wins.
